In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model


mobilenet_model = MobileNetV2(weights="imagenet", include_top=False)
model = load_model("path_to_your_face_mask_detection_model.h5")


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_and_predict_mask(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    faces_list = []
    preds = []
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  
        face = cv2.resize(face, (224, 224))
        face = preprocess_input(face)
        
        faces_list.append(face)
        
        if len(faces_list) > 0:
            preds = model.predict(np.array(faces_list))
    
    return faces, preds

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    faces, preds = detect_and_predict_mask(frame)
    
    for (face, pred) in zip(faces, preds):
        (mask, withoutMask) = pred
        
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        
        cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
    
    cv2.imshow("Face Mask Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'